In [ ]:
# The aim of this module is to calculate correlations for variables in the flat files
# the variables are then used for linear regression

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# I_FLAT = pd.read_pickle ('/Users/joezhou/Downloads/C_TICKER_FLAT.pkl') 
# C_BG_MODEL = pd.read_pickle ('/Users/joezhou/Downloads/C_BEN_G_MODEL.pkl') 

R_PRICE = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_Prices.pkl') 



In [2]:
# Find daily price correlations
# R_PRICE1 = R_PRICE[R_PRICE['TickName'].isin(['CBA.AX','ANZ.AX','WPC.AX','JBH.AX','LGP.AX'])]

R_PRICE_CLSE = R_PRICE[['Date', 'TickName','Close']]

R_PRICE_CLSE2 = R_PRICE_CLSE.pivot_table(index=['Date'], columns='TickName', values='Close').reset_index()

# Create correlation matrix
corr_matrix = R_PRICE_CLSE2.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper = upper.reset_index(level=0)
R_PRICE_CORR=upper.melt(id_vars=["TickName"],var_name="TickName2",value_name="CORR_PRICE").dropna()

R_PRICE_CORR_HIGH = R_PRICE_CORR.loc[(R_PRICE_CORR['CORR_PRICE'] >= 0.8) | (R_PRICE_CORR['CORR_PRICE'] <= -0.8)]


del R_PRICE_CLSE, R_PRICE_CLSE2, corr_matrix, upper

In [ ]:
# next iteration, loop differing time periods

# from datetime import datetime
# from dateutil.relativedelta import relativedelta

# yrs_ago1 = datetime.now() - relativedelta(years=1)
# yrs_ago3 = datetime.now() - relativedelta(years=3)
# yrs_ago5 = datetime.now() - relativedelta(years=5)
# yrs_ago10 = datetime.now() - relativedelta(years=10)

# PRICE_CALC = R_PRICE


# PRICE_CALC['DateTime'] = pd.to_datetime(PRICE_CALC['Date'], format='%Y-%m-%d')
# PRICE_CALC['Year'] = PRICE_CALC['DateTime'].dt.year

# PRICE_CALC['CALC_DIV_1YR'] = PRICE_CALC.loc[PRICE_CALC['DateTime'] >= yrs_ago1,['Dividends']].sum(axis=1)
# PRICE_CALC['CALC_DIV_3YR'] = PRICE_CALC.loc[PRICE_CALC['DateTime'] >= yrs_ago3,['Dividends']].sum(axis=1)
# PRICE_CALC['CALC_DIV_5YR'] = PRICE_CALC.loc[PRICE_CALC['DateTime'] >= yrs_ago5,['Dividends']].sum(axis=1)
# PRICE_CALC['CALC_DIV_10YR'] = PRICE_CALC.loc[PRICE_CALC
# ['DateTime'] >= yrs_ago10,['Dividends']].sum(axis=1)


In [3]:
R_PRICE_CORR.to_pickle("/Users/joezhou/Downloads/Investment/1 data/C_PRICE_CORR.pkl")
R_PRICE_CORR_HIGH.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_PRICE_CORR_HIGH.pkl")

# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/Investment/TEST_CORR_MATRIX{}.xlsx".format(dte)
# R_PRICE_CORR_HIGH.to_excel(fname)



In [16]:
# Bring in features to reduce set

DF_MASTER = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/I_MASTER_FLAT.pkl') 
DF_MASTER2 = DF_MASTER.add_suffix('2')

DF_MASTER_CORR = R_PRICE_CORR_HIGH.merge(DF_MASTER,how='inner',on='TickName').merge(DF_MASTER2,how='inner',on='TickName2')

# Apply filter conditions on the first ticker
DF_MASTER_CORR_ = DF_MASTER_CORR.loc[(DF_MASTER_CORR['CALC_BG_CONS_BG_SCORE'] >= 4) 
                                    & (DF_MASTER_CORR['CALC_BG_ENT_BG_SCORE'] >=4)
                                    & (DF_MASTER_CORR['CALC_PE_RATIO'] <=25)
                                    & (DF_MASTER_CORR['profitMargins'] >= 0.15)
                                    & (DF_MASTER_CORR['CALC_BG_CONS_BG_SCORE2'] >= 4) 
                                    & (DF_MASTER_CORR['CALC_BG_ENT_BG_SCORE2'] >=4)
                                    & (DF_MASTER_CORR['CALC_PE_RATIO2'] <=25)
                                    & (DF_MASTER_CORR['profitMargins2'] >= 0.15)
                                    ]

from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/Investment/TEST_CORR_MATRIX_HIGH_PAIRS{}.xlsx".format(dte)
DF_MASTER_CORR_.to_excel(fname)


In [26]:


# Note: code below is work in progress

In [62]:
# cluster to find similar performing

import matplotlib.pyplot as plt
# from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler


In [67]:
features, true_labels = make_blobs(
n_samples=200,
centers=3,
cluster_std=2.75,
random_state=42
 )

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

kmeans = KMeans(
   init="random",
   n_clusters=3,
   n_init=10,
   max_iter=300,
   random_state=42
   )


kmeans.fit(scaled_features)

KMeans(init='random', n_clusters=3, random_state=42)

In [ ]:
# Build sector related models
MOD_DATA = ADDED_PRICE_INF[['TickName','PROB_HIGH','CHG_RTE_PRICE','sector']]

# padding rows into columns of 1 or 0
# MOD_DATA = pd.concat([MOD_DATA, pd.get_dummies(MOD_DATA['sector'])], axis=1);


# transpose for excel movement correlations


MOD_DATA2 = MOD_DATA.pivot_table('PROB_HIGH', ['TickName','CHG_RTE_PRICE'], 'sector')

#Export version for Tableau

MOD_DATA2.to_excel('/Users/joezhou/Downloads/T_SECTOR_MODEL2.xls', index=False)


# del MOD_DATA['sector'], MOD_DATA['TickName']

# np.array(MOD_DATA, dtype=bool)


In [ ]:
# BG_MODEL.to_pickle("/Users/joezhou/Downloads/C_CALCS.pkl")
# BG_MODEL.to_excel('/Users/joezhou/Downloads/C_CALCS.xlsx', engine='xlsxwriter')  

